### onenet上传声音事件类型
准备条件：
 pip install paho-mqtt 

参数修改： 
client_id: 设备ID。  
username: 产品的ID，用于标识OneNet平台上的产品。  
accesskey: 设备的访问密钥，用于生成认证Token。  
event_data: 要发布到OneNet平台的数据  
等待时间：time.sleep(1)单位为s，可修改  


##仅onenet上传模拟数据

In [2]:
import paho.mqtt.client as mqtt
import json
import base64
import hmac
import time
from urllib.parse import quote

# MQTT Broker 配置
broker = "studio-mqtt.heclouds.com"
port = 1883

client_id = "camera"  # 从您的配置中获取
username = "ZBPW56xh7m"  # 从您的配置中获取
accesskey = "H1H/FdM3vRfraZNhsSed0We90/OjwX91E6OX529vdac="  # 请替换为您的实际访问密钥,产品密钥，需短信验证

# 认证token生成函数
def get_token(product_id, device_name, access_key):
    version = '2018-10-31'
    res = "products/" + product_id + "/devices/" + device_name
    et = str(int(time.time()) + 36000000)
    method = 'sha256'
    key = base64.b64decode(access_key)

    org = et + '\n' + method + '\n' + res + '\n' + version
    sign_b = hmac.new(key=key, msg=org.encode(), digestmod=method)
    sign = base64.b64encode(sign_b.digest()).decode()

    sign = quote(sign, safe='')
    res = quote(res, safe='')

    token = 'version=%s&res=%s&et=%s&method=%s&sign=%s' % (version, res, et, method, sign)

    return token

# 当客户端收到来自服务器的CONNACK响应时的回调
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully.")
        # 连接成功后就订阅topic
        client.subscribe("$sys/" + username + "/" + client_id + "/thing/event/post/reply", qos=1)
    else:
        print("Failed to connect, return code " + str(rc))

# 从服务器接收发布消息时的回调。
def on_message(client, userdata, message):
    print(f"Received message '{message.payload.decode()}' on topic '{message.topic}'")

# 创建 MQTT 客户端实例
client = mqtt.Client(client_id=client_id)
client.on_connect = on_connect
client.on_message = on_message

# 生成认证 Token
password = get_token(username, client_id, accesskey)
print("Generated token:", password)

# 连接到 MQTT 服务器
client.username_pw_set(username, password)
client.connect(broker, port, 60)

# 启动网络循环以处理连接
client.loop_start()

# 等待一段时间以完成连接测试
time.sleep(1)

# 构建暴力行为检测事件数据
start_time = int(time.time() * 1000)  # 时间戳（毫秒级）
confidence_value = 0.4  # 可信度（示例值）

event_data = {
    "id": "123",
    "version": "1.0",
    "params": {
        "camera_violence": {  # 对应图中标识符
            "value": {
                "time_violence": start_time,  # 暴力行为检测时间
                "confidence_violence": confidence_value  # 可信度
            },
            "time": start_time  # 事件时间
        }
    }
}

# 发布事件数据
topic = "$sys/" + username + "/" + client_id + "/thing/event/post"
client.publish(topic, json.dumps(event_data))
print(f"Published to {topic}: {event_data}")

# 等待一段时间以接收回复
time.sleep(1)

# 停止网络循环并断开连接
client.loop_stop()
client.disconnect()

C:\Users\hq134\AppData\Local\Temp\ipykernel_29276\1779066622.py:49: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=client_id)


Generated token: version=2018-10-31&res=products%2FZBPW56xh7m%2Fdevices%2Fcamera&et=1782946230&method=sha256&sign=5Y%2BRftEDAUfhr5BFKfvXR%2Fp7eNrHP1dk8AQ%2BjG1qBFc%3D
Connected successfully.
Published to $sys/ZBPW56xh7m/camera/thing/event/post: {'id': '123', 'version': '1.0', 'params': {'camera_violence': {'value': {'time_violence': 1746946231699, 'confidence_violence': 0.4}, 'time': 1746946231699}}}
Received message '{"id":"123","code":200,"msg":"success"}' on topic '$sys/ZBPW56xh7m/camera/thing/event/post/reply'


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

## 仅仅实时视频检测暴力行为

In [1]:
import cv2
import numpy as np
from collections import deque
from keras.models import load_model
import os

# ======================
# 配置与常量定义
# ======================
os.environ['FFMPEG_LOGLEVEL'] = 'verbose'  # 启用 FFmpeg 详细日志（显示解码错误）

# 模型与视频参数
IMG_SIZE = 128                # 输入模型的图像尺寸
FRAME_COUNT = 15              # 积累帧数用于时间序列预测
CLASS_NAMES = ['No Violence', 'Violence']  # 分类标签
CONFIDENCE_THRESHOLD = 0.5    # 置信度阈值
QUEUE_SIZE = 10               # 滑动平均队列大小
RTSP_URL = 'http://192.168.137.231/mjpeg/1'    # 请替换为你的 RTSP 流地址
#rtmp://47.94.236.247/live/stream
MODEL_PATH ='D:/jnu/物联网设计/视频处理/Real-Time-Violence-Detection-main/bestt.keras'    # 模型路径

# ======================
# 模型加载
# ======================
try:
    model = load_model(MODEL_PATH)
    print(f"[INFO] 模型加载成功: {MODEL_PATH}")
except Exception as e:
    print(f"[ERROR] 模型加载失败: {str(e)}")
    exit(1)

# ======================
# 帧预处理函数
# ======================
def preprocess_frame(frame, size=(IMG_SIZE, IMG_SIZE)):
    """预处理帧：缩放、BGR转RGB、归一化"""
    frame = cv2.resize(frame, size)
    frame = frame[:, :, [2, 1, 0]]  # OpenCV BGR转RGB
    frame = frame / 255.0  # 归一化到 [0, 1]
    return frame

# ======================
# 预测后处理函数
# ======================
def smooth_predictions(prediction, queue):
    """滑动平均平滑预测结果"""
    queue.append(prediction)
    return np.mean(queue)

def majority_voting(queue, threshold=CONFIDENCE_THRESHOLD):
    """多数投票：超过阈值的预测占比是否过半"""
    count = sum(1 for p in queue if p > threshold)
    return count / len(queue) > threshold

# ======================
# 主函数：实时视频流处理
# ======================
def classify_live_stream():
    cap = cv2.VideoCapture(RTSP_URL)
    
    if not cap.isOpened():
        print(f"[ERROR] 无法打开 RTSP 流: {RTSP_URL}")
        print("[提示] 请检查 RTSP 地址是否正确，设备是否在线")
        exit(1)

    print(f"[INFO] 已连接到 RTSP 流，按 'q' 退出")
    frames = []                  # 待预测的帧序列
    prediction_queue = deque(maxlen=QUEUE_SIZE)  # 预测结果队列

    while True:
        # 1. 读取视频帧
        ret, frame = cap.read()
        
        # 2. 处理帧读取失败（RTSP 流常见问题）
        if not ret:
            print("[WARNING] 读取帧失败，正在重新连接...")
            
            # 释放资源并重新连接
            cap.release()
            cap = cv2.VideoCapture(RTSP_URL)
            if not cap.isOpened():
                print("[ERROR] 重新连接失败，继续重试...")
                continue
            continue

        try:
            display_frame = cv2.resize(frame, (800, 600))  # 调整显示窗口大小
            processed_frame = preprocess_frame(frame)
            frames.append(processed_frame)

            is_violent = False          # 初始化默认状态：安全
            display_text = "No Violence"  # 默认显示文本
            
            # 3. 积累足够帧后进行预测
            if len(frames) == FRAME_COUNT:
                input_array = np.expand_dims(np.array(frames), axis=0)
                prediction = model.predict(input_array)[0][0]  # 获取预测概率（0-1）
                smoothed_pred = smooth_predictions(prediction, prediction_queue)
                
                # 基础判断：超过置信度阈值
                is_violent = smoothed_pred > CONFIDENCE_THRESHOLD
                # 多数投票（队列满时启用，减少抖动）
                if len(prediction_queue) == QUEUE_SIZE:
                    is_violent = majority_voting(prediction_queue)
                
                # 生成带置信度的标签
                confidence = f"{smoothed_pred:.2f}"
                display_text = (
                    f"Violence (Confidence: {confidence})" 
                    if is_violent 
                    else "No Violence"
                )

                frames = []  # 清空帧列表，等待下一周期

            # 4. 在视频帧上绘制标签（颜色区分状态：绿色=安全，红色=暴力）
            color = (0, 255, 0) if not is_violent else (0, 0, 255)
            cv2.putText(
                display_frame,
                display_text,
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                color,
                2,
                cv2.LINE_AA
            )

            # 5. 显示视频帧
            cv2.imshow('Real-Time Violence Detection', display_frame)

        except Exception as e:
            # 捕获 Python 异常（如模型输入错误、数组操作错误）
            print(f"[ERROR] 帧处理失败: {str(e)}")
            if frame is not None:
                print(f"[DEBUG] 帧尺寸: {frame.shape}, 数据类型: {frame.dtype}")
            continue  # 跳过当前错误帧，继续处理下一帧

        # 6. 退出逻辑（按 'q' 键）
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 释放资源
    cap.release()
    cv2.destroyAllWindows()
    print("[INFO] 程序已退出")

# ======================
# 程序入口
# ======================
if __name__ == "__main__":
    classify_live_stream()

[INFO] 模型加载成功: D:/jnu/物联网设计/视频处理/Real-Time-Violence-Detection-main/bestt.keras
[INFO] 已连接到 RTSP 流，按 'q' 退出
1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step
[WARNING] 读取帧失败，正在重新连接...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70m

## 实时检测+上传onenet：
由于onenet代码里设置了1s的等待回复，且不想一直与onenet保持连接，所以设置多线程.

In [ ]:
import paho.mqtt.client as mqtt
import json
import base64
import hmac
import time
from urllib.parse import quote
import cv2
import numpy as np
from collections import deque
from keras.models import load_model
import os
import threading

# ======================
# 配置与常量定义
# ======================
os.environ['FFMPEG_LOGLEVEL'] = 'verbose'
IMG_SIZE = 128
FRAME_COUNT = 15
CLASS_NAMES = ['No Violence', 'Violence']
CONFIDENCE_THRESHOLD = 0.5
QUEUE_SIZE = 10
RTSP_URL = 'http://192.168.137.165/mjpeg/1'
MODEL_PATH = 'D:/jnu/物联网设计/视频处理/Real-Time-Violence-Detection-main/bestt.keras'

# ======================
# OneNET 配置
# ======================
broker = "studio-mqtt.heclouds.com"
port = 1883
client_id = "camera"
username = "ZBPW56xh7m"
accesskey = "H1H/FdM3vRfraZNhsSed0We90/OjwX91E6OX529vdac="

# ======================
# OneNET 核心函数
# ======================
def get_token(product_id, device_name, access_key):
    version = '2018-10-31'
    res = f"products/{product_id}/devices/{device_name}"
    et = str(int(time.time()) + 36000000)
    method = 'sha256'
    key = base64.b64decode(access_key)
    org = f"{et}\n{method}\n{res}\n{version}"
    sign_b = hmac.new(key=key, msg=org.encode(), digestmod=method)
    sign = base64.b64encode(sign_b.digest()).decode()
    sign = quote(sign, safe='')
    res = quote(res, safe='')
    return f'version={version}&res={res}&et={et}&method={method}&sign={sign}'

def upload_violence_to_onenet(confidence):
    client = mqtt.Client(client_id=client_id)
    
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected successfully.")
            client.subscribe(f"$sys/{username}/{client_id}/thing/event/post/reply", qos=1)
        else:
            print(f"Failed to connect, return code {rc}")
    
    def on_message(client, userdata, message):
        print(f"Received message: {message.payload.decode()}")
    
    client.on_connect = on_connect
    client.on_message = on_message
    password = get_token(username, client_id, accesskey)
    client.username_pw_set(username, password)
    client.connect(broker, port, 60)
    client.loop_start()
    time.sleep(0.5)
    
    event_data = {
        "id": "123",
        "version": "1.0",
        "params": {
            "camera_violence": {
                "value": {
                    "time_violence": int(time.time() * 1000),
                    "confidence_violence": round(float(confidence), 2)
                },
                "time": int(time.time() * 1000)
            }
        }
    }
    client.publish(f"$sys/{username}/{client_id}/thing/event/post", json.dumps(event_data))
    print(f"Uploaded: {event_data}")
    time.sleep(1)
    client.loop_stop()
    client.disconnect()

# ======================
# 视频检测核心函数（含平滑处理）
# ======================
def preprocess_frame(frame):
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame = frame[:, :, [2, 1, 0]]  # BGR转RGB
    frame = frame / 255.0  # 归一化
    return frame

def smooth_predictions(prediction, queue):
    queue.append(prediction)
    return np.mean(queue)

def majority_voting(queue, threshold=CONFIDENCE_THRESHOLD):
    count = sum(1 for p in queue if p > threshold)
    return count / len(queue) > threshold

def classify_live_stream():
    model = load_model(MODEL_PATH)
    cap = cv2.VideoCapture(RTSP_URL)
    if not cap.isOpened():
        print(f"Error: Cannot open RTSP stream {RTSP_URL}")
        return
    
    frames = []
    prediction_queue = deque(maxlen=QUEUE_SIZE)
    last_upload_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Warning: Failed to read frame, reconnecting...")
            cap.release()
            cap = cv2.VideoCapture(RTSP_URL)
            continue
        
        display_frame = cv2.resize(frame, (800, 600))
        processed_frame = preprocess_frame(frame)
        frames.append(processed_frame)
        
        is_violent = False
        display_text = "No Violence"  # 默认不显示置信度
        
        # 仅在积累足够帧时进行预测
        if len(frames) == FRAME_COUNT:
            input_array = np.expand_dims(np.array(frames), axis=0)
            prediction = model.predict(input_array)[0][0]
            smoothed_pred = smooth_predictions(prediction, prediction_queue)  # 滑动平均
            is_violent = smoothed_pred > CONFIDENCE_THRESHOLD
            
            # 多数投票（队列满时启用）
            if len(prediction_queue) == QUEUE_SIZE:
                is_violent = majority_voting(prediction_queue)
            
            confidence = f"{smoothed_pred:.2f}"
            display_text = (
                f"Violence (Confidence: {confidence})" 
                if is_violent 
                else "No Violence"
            )
            frames = []  # 清空帧队列
            
            # 上传逻辑（示例：可添加间隔控制）
            if is_violent and (time.time() - last_upload_time) >= 5:
                threading.Thread(target=upload_violence_to_onenet, args=(smoothed_pred,)).start()
                last_upload_time = time.time()
        
        # 绘制文本（确保smoothed_pred仅在预测时存在）
        color = (0, 255, 0) if not is_violent else (0, 0, 255)
        cv2.putText(
            display_frame,
            display_text,
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            color,
            2,
            cv2.LINE_AA
        )
        
        cv2.imshow('Real-Time Violence Detection', display_frame)
        if cv2.waitKey(1) == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# ======================
# 程序入口
# ======================
if __name__ == "__main__":
    classify_live_stream()